<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=76eb7cd8d2c82ee0cf67665daefbb85b69b4baef42d7b4675d2aa39244d47be6
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-09 10:25:26
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15


-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  4.95 L
Current:  1.38 C
-------------------
Today PnL: -86.61 K (-0.62%)
Current PnL: -30.26 L (-19.55%)
CY Booked + Current PnL: -15.97 L (-10.32%)
-------------------
Total profit:  1.06 L
Total loss:  -31.32 L
-------------------
Total Booked + Current PnL: 11.01 L (8.61%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.35%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.33 C
Est FTT PnL: 94.89 L (68.76%)
Deployed:  1.28 C
Current:  1.38 C
CAGR/XIRR %: 4.3%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,-0.55,-18.10,22.21,0.09,18331.0,-18240.0,82537.0,91.68,31.0,M-SC,2.19,253.0,-1.00,0.62,8.43,OX40N,NTT,DURABLES
43,ITC,452.00,0.03,-1.68,12.33,10.44,24262.0,-3364.0,196774.0,-41.15,45.0,X-LC,1.65,5.0,-0.14,1.48,3.79,X40,NTT,FMCG
50,MASFIN,398.61,-0.23,-4.98,28.44,22.05,26479.0,-4875.0,93105.0,-18.25,49.0,H-SC,6.92,164.0,-0.18,0.70,35.33,XR,ATH,FINANCE
39,INDIGOPNTS,1408.00,-1.01,-15.35,18.16,0.02,26833.0,-26801.0,147758.0,125.45,45.0,M-SC,16.70,234.0,-1.00,1.11,28.52,OX40N,NTT,PAINTS
35,ICICIGI,2252.93,-1.10,3.47,16.16,20.19,29148.0,6054.0,180374.0,-18.85,39.0,X-MC,5.95,68.0,0.21,1.36,19.11,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,1.01,-38.20,118.77,35.20,94448.0,-49148.0,79522.0,4.26,28.0,X-SC,15.28,92.0,-0.52,0.60,1.01,X40,NTT,FOOTWEAR
42,IRCTC,1269.00,0.95,-19.54,90.08,52.93,196048.0,-52870.0,217638.0,-40.53,34.0,H-MC,5.41,118.0,-0.27,1.64,0.95,XY24,BTT,TRAVEL
29,HAPPSTMNDS,1480.71,0.94,-38.05,198.38,84.85,157514.0,-48765.0,79400.0,-26.35,45.0,H-SC,13.64,147.0,-0.31,0.60,1.91,AR,ATH,IT
10,BANDHANBNK,400.00,0.94,-34.22,192.50,92.42,352747.0,-95314.0,183245.0,28.59,25.0,H-SC,7.98,170.0,-0.27,1.38,5.90,XY24,NTT,BANKS
36,ICICIPRULI,790.00,0.88,3.07,27.07,30.97,49310.0,5420.0,182158.0,-21.15,54.0,X-MC,2.04,75.0,0.11,1.37,16.01,X40,ATH,INSURANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COFFEEDAY,80.00,-4.64,-48.55,162.90,35.27,95176.0,-55123.0,58426.0,-59.76,18.0,L-SC,12.34,270.0,-0.58,0.44,43.00,XR,NTT,HOTELS
5,ASIANPAINT,3460.25,-4.14,-1.88,23.26,20.95,57460.0,-4734.0,247034.0,-2.88,50.0,X-LC,19.04,31.0,-0.08,1.86,33.09,X40,ATH,PAINTS
28,GREENPANEL,537.00,-2.86,-39.76,137.61,43.14,151767.0,-72790.0,110288.0,146.77,22.0,M-SC,8.67,240.0,-0.48,0.83,3.74,XY24,NTT,MISC
6,ASIANTILES,137.00,-2.14,-21.18,130.48,81.67,96481.0,-19867.0,73943.0,6638.89,28.0,L-SC,14.77,271.0,-0.21,0.56,43.19,XR,NTT,CERAMICS
24,EASEMYTRIP,26.40,-2.05,-57.84,245.55,45.70,183060.0,-102264.0,74551.0,-22.68,48.0,M-SC,23.65,205.0,-0.56,0.56,7.45,XY24,NTT,TRAVEL


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.42,-9.54,129.84,107.91,173408.0,-14089.0,133555.0,-26.23,23.0,M-SC,11.94,216.0,-0.08,1.00,0.00,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-1.01,-15.35,18.16,0.02,26833.0,-26801.0,147758.0,125.45,45.0,M-SC,16.70,234.0,-1.00,1.11,28.52,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,-0.55,-18.10,22.21,0.09,18331.0,-18240.0,82537.0,91.68,31.0,M-SC,2.19,253.0,-1.00,0.62,8.43,OX40N,NTT,DURABLES
66,SIS,528.00,-0.29,-25.39,62.54,21.26,51806.0,-28195.0,82837.0,1952.85,43.0,H-SC,4.63,166.0,-0.54,0.62,11.92,OX40N,NTT,MISC
47,KANSAINER,340.00,0.15,-25.85,53.03,13.47,106040.0,-69705.0,199962.0,-69.48,26.0,H-SC,3.21,158.0,-0.66,1.50,1.67,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,-1.5,5.59,63.14,72.27,112975.0,9480.0,178928.0,-9.26,64.0,M-LC,2.23,99.0,0.08,1.34,12.12,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,-1.50,5.59,63.14,72.27,112975.0,9480.0,178928.0,-9.26,64.0,M-LC,2.23,99.0,0.08,1.34,12.12,XR,NTT,IT
38,INDIAMART,4810.62,-1.04,-3.32,113.82,106.72,135718.0,-4097.0,119239.0,-52.15,32.0,H-SC,1.18,139.0,-0.03,0.90,19.28,AR,ATH,MISC
85,ZYDUSLIFE,1286.17,-0.35,-2.84,39.60,35.63,80633.0,-5956.0,203618.0,-16.74,36.0,H-MC,3.16,119.0,-0.07,1.53,13.36,AR,ATH,PHARMA
37,IEX,219.00,-0.49,-4.16,55.08,48.63,105553.0,-8318.0,191636.0,-36.24,48.0,H-SC,14.09,136.0,-0.08,1.44,6.91,XR,NTT,MISC
25,FINCABLES,1641.55,-0.42,-9.54,129.84,107.91,173408.0,-14089.0,133555.0,-26.23,23.0,M-SC,11.94,216.0,-0.08,1.00,0.00,OX40N,ATH,CABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,-0.18,-23.54,94.16,48.45,144354.0,-47199.0,153307.0,-29.11,10.0,X-MC,11.26,64.0,-0.33,1.15,0.00,X40N,ATH,FINANCE
22,DIXON,18931.72,-0.76,-13.56,40.65,21.58,49243.0,-18999.0,121140.0,-56.42,24.0,X-MC,5.47,56.0,-0.39,0.91,7.25,X40N,ATH,IT
46,JSWINFRA,342.00,-0.42,-11.78,30.61,15.23,54103.0,-23598.0,176749.0,-24.50,25.0,X-MC,7.93,66.0,-0.44,1.33,16.67,X40N,NTT,REALTY
75,TMPV,600.00,-0.46,-17.84,73.06,42.18,165910.0,-49322.0,227088.0,-26.74,26.0,X-LC,5.07,3.0,-0.30,1.71,0.00,XY24,NTT,AUTO
11,BATAINDIA,2096.00,1.01,-38.20,118.77,35.20,94448.0,-49148.0,79522.0,4.26,28.0,X-SC,15.28,92.0,-0.52,0.60,1.01,X40,NTT,FOOTWEAR


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,AWL,485.0,-1.28,-21.70,96.04,53.50,226911.0,-65484.0,236267.0,-62.10,32.0,X-MC,1.25,58.0,-0.29,1.78,4.94,XY24,NTT,FMCG
43,ITC,452.0,0.03,-1.68,12.33,10.44,24262.0,-3364.0,196774.0,-41.15,45.0,X-LC,1.65,5.0,-0.14,1.48,3.79,X40,NTT,FMCG
3,ACC,3906.0,-0.32,-24.77,118.19,64.15,211584.0,-58931.0,179020.0,-55.88,36.0,X-SC,1.78,82.0,-0.28,1.35,0.42,XY24,BTT,CEMENT
15,CAMS,4762.0,-0.73,-80.40,536.72,24.79,264930.0,-202495.0,49361.0,-81.32,39.0,X-SC,1.84,86.0,-0.76,0.37,20.67,X40N,NTT,MISC
36,ICICIPRULI,790.0,0.88,3.07,27.07,30.97,49310.0,5420.0,182158.0,-21.15,54.0,X-MC,2.04,75.0,0.11,1.37,16.01,X40,ATH,INSURANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,-0.18,-23.54,94.16,48.45,144354.0,-47199.0,153307.0,-29.11,10.0,X-MC,11.26,64.0,-0.33,1.15,0.0,X40N,ATH,FINANCE
53,PAGEIND,51605.08,-0.75,-9.72,39.64,26.07,58596.0,-15920.0,147820.0,-33.51,29.0,X-MC,11.51,55.0,-0.27,1.11,0.0,X40,ATH,APPARELS
59,RELAXO,1176.00,-0.16,-48.56,196.07,52.31,146413.0,-70486.0,74674.0,-45.21,30.0,X-SC,6.17,91.0,-0.48,0.56,0.0,X40N,NTT,FOOTWEAR
55,QUESS,424.00,-1.91,-32.36,111.18,42.84,48887.0,-21035.0,43971.0,-55.92,34.0,X-SC,37.13,83.0,-0.43,0.33,0.0,XY24,NTT,MISC
54,PGHH,17907.65,-1.15,-6.04,42.35,33.76,79914.0,-12120.0,188700.0,-33.72,33.0,X-MC,5.10,57.0,-0.15,1.42,0.0,X40,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,-1.91,-32.36,111.18,42.84,48887.0,-21035.0,43971.0,-55.92,34.0,X-SC,37.13,83.0,-0.43,0.33,0.00,XY24,NTT,MISC
15,CAMS,4762.00,-0.73,-80.40,536.72,24.79,264930.0,-202495.0,49361.0,-81.32,39.0,X-SC,1.84,86.0,-0.76,0.37,20.67,X40N,NTT,MISC
59,RELAXO,1176.00,-0.16,-48.56,196.07,52.31,146413.0,-70486.0,74674.0,-45.21,30.0,X-SC,6.17,91.0,-0.48,0.56,0.00,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,1.01,-38.20,118.77,35.20,94448.0,-49148.0,79522.0,4.26,28.0,X-SC,15.28,92.0,-0.52,0.60,1.01,X40,NTT,FOOTWEAR
22,DIXON,18931.72,-0.76,-13.56,40.65,21.58,49243.0,-18999.0,121140.0,-56.42,24.0,X-MC,5.47,56.0,-0.39,0.91,7.25,X40N,ATH,IT


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,-1.56,-13.44,37.79,19.28,113172.0,-46481.0,299475.0,-24.60,59.0,X-LC,5.78,1.0,-0.41,2.25,10.71,X40,ATH,IT
41,INFY,2275.00,-1.40,8.12,43.24,54.87,146275.0,25399.0,338287.0,-15.65,61.0,X-LC,3.36,2.0,0.17,2.54,17.07,X40,BTT,IT
75,TMPV,600.00,-0.46,-17.84,73.06,42.18,165910.0,-49322.0,227088.0,-26.74,26.0,X-LC,5.07,3.0,-0.30,1.71,0.00,XY24,NTT,AUTO
81,VBL,671.64,-1.30,-6.33,44.84,35.67,132655.0,-20001.0,295841.0,-17.49,47.0,X-LC,4.84,4.0,-0.15,2.22,6.64,X40N,ATH,FMCG
43,ITC,452.00,0.03,-1.68,12.33,10.44,24262.0,-3364.0,196774.0,-41.15,45.0,X-LC,1.65,5.0,-0.14,1.48,3.79,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,MASFIN,398.61,-0.23,-4.98,28.44,22.05,26479.0,-4875.0,93105.0,-18.25,49.0,H-SC,6.92,164.0,-0.18,0.70,35.33,XR,ATH,FINANCE
13,BSOFT,831.70,-0.20,-22.53,94.64,50.79,103122.0,-31691.0,108962.0,-0.66,69.0,H-SC,5.82,151.0,-0.31,0.82,26.87,XR,ATH,IT
39,INDIGOPNTS,1408.00,-1.01,-15.35,18.16,0.02,26833.0,-26801.0,147758.0,125.45,45.0,M-SC,16.70,234.0,-1.00,1.11,28.52,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.25,-4.14,-1.88,23.26,20.95,57460.0,-4734.0,247034.0,-2.88,50.0,X-LC,19.04,31.0,-0.08,1.86,33.09,X40,ATH,PAINTS
68,STARHEALTH,761.00,0.50,-10.21,64.01,47.26,161571.0,-28707.0,252416.0,22.63,36.0,H-SC,13.21,171.0,-0.18,1.90,35.24,XY24,NTT,INSURANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BSOFT,831.70,-0.20,-22.53,94.64,50.79,103122.0,-31691.0,108962.0,-0.66,69.0,H-SC,5.82,151.0,-0.31,0.82,26.87,XR,ATH,IT
32,HCLTECH,1922.01,-0.87,8.63,14.82,24.73,38947.0,20878.0,262802.0,13.61,67.0,X-LC,5.80,7.0,0.54,1.98,25.18,X40,ATH,IT
84,WIPRO,420.00,-1.50,5.59,63.14,72.27,112975.0,9480.0,178928.0,-9.26,64.0,M-LC,2.23,99.0,0.08,1.34,12.12,XR,NTT,IT
5,ASIANPAINT,3460.25,-4.14,-1.88,23.26,20.95,57460.0,-4734.0,247034.0,-2.88,50.0,X-LC,19.04,31.0,-0.08,1.86,33.09,X40,ATH,PAINTS
73,TCS,4389.97,-1.56,-13.44,37.79,19.28,113172.0,-46481.0,299475.0,-24.60,59.0,X-LC,5.78,1.0,-0.41,2.25,10.71,X40,ATH,IT


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.84
1,25,45.87
2,50,76.81


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.80
MC    30.39
LC    24.83
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.30
X40      22.59
X40N     10.86
XR       10.00
XY25      9.44
AR        9.37
OX40N     7.69
SR        1.02
MH        0.75
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.44
X-MC    23.62
X-LC    21.25
M-SC    12.01
X-SC     6.99
H-MC     5.00
M-MC     1.41
L-SC     1.36
M-LC     1.34
H-LC     1.19
L-LC     1.05
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.51,-8.11,43.07
IT,12.88,-17.56,81.11
FINANCE,9.49,-22.66,74.49
ELECTRICAL,6.27,-12.55,53.78
MISC,6.13,-59.48,124.81
PAINTS,6.08,-14.07,31.01
INSURANCE,4.63,-2.80,39.03
PHARMA,4.10,-2.38,35.14
AUTO,3.44,-25.48,79.28


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3271651.0,21
XR,1332279.0,13
AR,1326679.0,10
X40,1034486.0,14
X40N,1002086.0,9
OX40N,767461.0,10
XY25,385651.0,6
SR,285209.0,2
MH,83818.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3727604.0,25
M-SC,1459086.0,15
X-MC,1340547.0,16
X-SC,989822.0,8
X-LC,935842.0,11
H-MC,414500.0,3
L-SC,280614.0,3
M-LC,112975.0,1
H-LC,75839.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1309113.0      6
           AR         904264.0      5
M-SC       XY24       824744.0      6
H-SC       XR         794665.0      7
X-SC       X40N       507249.0      3
X-MC       X40        467030.0      7
           XY24       399841.0      3
X-LC       X40        380116.0      5
H-SC       OX40N      350535.0      4
M-SC       OX40N      327969.0      5
X-SC       XY24       295233.0      3
X-MC       X40N       285287.0      4
H-SC       SR         285209.0      2
X-LC       XY24       246672.0      2
H-MC       AR         218452.0      2
X-LC       X40N       209550.0      2
H-MC       XY24       196048.0      1
L-SC       XR         191657.0      2
X-MC       XY25       188389.0      2
X-SC       X40        187340.0      2
M-SC       XR         178249.0      2
           AR         128124.0      2
M-LC       XR         112975.0      1
X-LC       XY25        99504.0      2
L-SC       OX40N       88957.0      1
H-SC       MH          83818.0      1
H-LC       AR          75839.0      1
M-MC       XY25        57474.0      1
L-MC       XR          54733.0      1
L-LC       XY25        40284.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 42.0 seconds
